In [1]:
import re

import uproot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
keys = [
    "C1p,I1p,I1p",
    "C1p,I2p,I1p", "C1p,I2p,I2p",
    "C1p,I3p,I1p", "C1p,I3p,I2p", "C1p,I3p,I3p",
    "C1p,I4p,I2p", "C1p,I4p,I3p", "C1p,I4p,I4p",
    "C2p,I1p,I1p",
    "C2p,I2p,I1p", "C2p,I2p,I2p",
    "C2p,I3p,I1p", "C2p,I3p,I2p", "C2p,I3p,I3p",
    "C2p,I4p,I2p", "C2p,I4p,I3p", "C2p,I4p,I4p", 
    "C2p,I5p,I3p", "C2p,I5p,I4p", "C2p,I5p,I5p",
    "C3p,I2p,I2p",
    "C3p,I3p,I2p", "C3p,I3p,I3p",
    "C3p,I4p,I2p", "C3p,I4p,I3p", "C3p,I4p,I4p",
    "C3p,I5p,I2p", "C3p,I5p,I3p", "C3p,I5p,I4p", "C3p,I5p,I5p",
    "C3p,I6p,I3p", "C3p,I6p,I4p", "C3p,I6p,I5p", "C3p,I6p,I6p",
    "C3p,I7p,I4p", "C3p,I7p,I5p", "C3p,I7p,I6p", "C3p,I7p,I7p",
    "C3p,I8p,I5p", "C3p,I8p,I6p", "C3p,I8p,I7p",
    "C4p,I7p,I5p", "C4p,I7p,I6p", "C4p,I7p,I7p",
    "C4p,I8p,I5p", "C4p,I8p,I6p", "C4p,I8p,I7p",
]
filenames = [
    "/helium/analysis/SACLA2015B/analysis/CH2I2/Analysis_aq50-58LPS_r18_v3-CII-XFELonly_tuned_Angle=150_SelectThetaZ=120-180_MomSumfixed_C1+C2+.root",
    "/helium/analysis/SACLA2015B/analysis/CH2I2/Analysis_aq50-58LPS_r18_v3-CII-XFELonly_tuned_Angle=150_SelectThetaZ=120-180_MomSumfixed_C3+C4+.root",
]

In [8]:
def gen():
    for k in keys:
        charges = {
            nth: int(v)
            for nth, v in zip(["1st q", "2nd q", "3rd q"], re.search("C([1-9])p,I([1-9])p,I([1-9])p", k).groups())
        }
        for fn in filenames:
            with uproot.open(fn) as fp:
                if k.replace(",", "") not in fp:
                    continue
                hist = fp["{}/KE/KE_Sum".format(k.replace(",", ""))]
            if hist.values.sum() == 0:
                yield {
                    "Target": k,
                    "Count": 0,
                    **charges,
                }
                continue
            x = (hist.edges[:-1] + hist.edges[1:]) / 2
            avg = np.average(x, weights=hist.values)
            std = np.average((x - avg) ** 2, weights=hist.values) ** 0.5
            yield {
                "Target": k,
                "Count": hist.values.astype("int").sum(),
                "Where max (eV)": x[hist.values.argmax()],
                "Value (eV)": avg,
                "Std (eV)": std,
                **charges,
            }


df = pd.DataFrame(list(gen()))
df.head()

,1st q,2nd q,3rd q,Count,Std (eV),Target,Value (eV),Where max (eV)
0,1,1,1,64,7.206700,"C1p,I1p,I1p",20.968750,15.5
1,1,2,1,271,7.337994,"C1p,I2p,I1p",30.400369,26.5
2,1,2,2,155,11.137241,"C1p,I2p,I2p",45.583871,37.5
3,1,3,1,70,11.335595,"C1p,I3p,I1p",46.200000,44.5
4,1,3,2,378,12.487648,"C1p,I3p,I2p",61.444444,57.5


In [11]:
def gen():
    for k in keys:
        charges = {
            nth: int(v)
            for nth, v in zip(["1st q", "2nd q", "3rd q"], re.search("C([1-9])p,I([1-9])p,I([1-9])p", k).groups())
        }
        for fn in filenames:
            with uproot.open(fn) as fp:
                if k.replace(",", "") not in fp:
                    continue

            for which, h in [
                    ["btw 1st and 2nd", "{}/FormedAngle/AngleCI1CondXYZ"],
                    ["btw 1st and 3rd", "{}/FormedAngle/AngleCI2CondXYZ"],
                    ["btw 2nd and 3rd", "{}/FormedAngle/AngleIICondXYZ"],
                ]:
                with uproot.open(fn) as fp:
                    hist = fp[h.format(k.replace(",", ""))]
                if hist.values.sum() == 0:
                    yield {
                        "Target": k,
                        "Which angle": which,
                        "Count": 0,
                        **charges,
                    }
                    continue
                x = (hist.edges[:-1] + hist.edges[1:]) / 2
                avg = np.average(x, weights=hist.values)
                std = np.average((x - avg) ** 2, weights=hist.values) ** 0.5
                yield {
                    "Target": k,
                    "Which angle": which,
                    "Count": hist.values.astype("int").sum(),
                    "Value (deg)": avg,
                    "Std (deg)": std,
                    "Derivative": 1 / np.sin(avg / 180 * np.pi),
                    **charges,
                }

        
df = pd.DataFrame(list(gen()))
df.head()

,1st q,2nd q,3rd q,Count,Derivative,Std (deg),Target,Value (deg),Which angle
0,1,1,1,64,1.027349,10.616909,"C1p,I1p,I1p",103.250000,btw 1st and 2nd
1,1,1,1,64,1.121393,10.647357,"C1p,I1p,I1p",116.906250,btw 1st and 3rd
2,1,1,1,64,1.529538,8.139515,"C1p,I1p,I1p",139.171875,btw 2nd and 3rd
3,1,2,1,271,1.077153,9.107944,"C1p,I2p,I1p",111.817343,btw 1st and 2nd
4,1,2,1,271,1.039431,9.290338,"C1p,I2p,I1p",105.832103,btw 1st and 3rd
